# Tracing With LangSmith

In [1]:
%%capture
!pip install -U langsmith
!pip install --upgrade -q openai langchain langchain-openai langchain-community langgraph

In [2]:
import os
import openai
import getpass
# import logging

# logging.basicConfig(level=logging.DEBUG)


In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LANGCHAIN_API_KEY: ")

Enter your OPENAI_API_KEY:  ········
Enter your LANGCHAIN_API_KEY:  ········


In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "pr-tracing-with-langsmith"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [8]:
from langsmith import wrappers, traceable

# Auto-trace LLM calls in-context
client = wrappers.wrap_openai(openai.Client())


@traceable
def format_prompt(user_prompt):
    return [
        {
            'role': 'system',
            'content': 'You are a helpful assistant.',
        },
        {
            'role': 'user',
            'content': f'Generate three good names for an online store that sells {user_prompt}?'
        }
    ]


@traceable(run_type='llm')
def invoke_llm(messages):
    return client.chat.completions.create(
        messages=messages, model='gpt-4o-mini', temperature=0
    )


@traceable
def parse_output(response):
    return response.choices[0].message.content


@traceable
def run_pipeline():
    messages = format_prompt('Men\'s Soccer Boats')
    response = invoke_llm(messages)
    return parse_output(response)
    
result = run_pipeline()

In [9]:
from IPython.display import Markdown, display

display(Markdown(result))

Here are three name suggestions for your online store that sells men's soccer boots:

1. **Kick & Style** - This name emphasizes both the action of kicking and the stylish aspect of soccer boots.

2. **Goal Gear** - A catchy name that highlights the purpose of the boots while suggesting a range of soccer-related products.

3. **Striker's Choice** - This name appeals to players who want to make a statement on the field, suggesting that your store offers top-quality options for strikers and soccer enthusiasts alike.

## Tracing the Reflection Agentic App with LangSmith

In [10]:
os.environ["LANGCHAIN_PROJECT"] = "pr-langGraph-reflection"

In [11]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

In [12]:
generation_prompt = ChatPromptTemplate.from_messages([
    (
        'system',
        '''You are a Twitter expert assigned to craft outstanding tweets.
        Generate the most engaging and impactful tweet possible based on the user's request.
        If the user provides feedback, refine and enhance your previous attempts accordingly for maximum engagement.''',
    ),
    MessagesPlaceholder(variable_name='messages'),
])
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)
# using LCEL to create the generate_chain
generate_chain  = generation_prompt | llm

tweet = ''
request = HumanMessage(
    content='FIFA World Cup 26'
)

for chunk in generate_chain.stream(
    {'messages': [request]}
): 
    print(chunk.content, end="")
    tweet += chunk.content

🌍⚽️ The countdown to #FIFAWorldCup26 has begun! 🏆 With matches set to light up North America, which team are you rooting for? 🇺🇸🇨🇦🇲🇽 Let's make history together! 🎉 Share your predictions below! 👇 #WorldCup2026 #SoccerFever

In [13]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            '''You are a Twitter influencer known for your engaging content and sharp insights.
            Review and critique the user’s tweet.
            Provide constructive feedback, focusing on enhancing its depth, style, and overall impact.
            Offer specific suggestions to make the tweet more compelling and engaging for their audience.'''
        ),
        MessagesPlaceholder(variable_name='messages'),
    ]
)

reflect_chain  = reflection_prompt | llm

reflection = ''
# streaming the response
for chunk in reflect_chain.stream(
    {'messages': [request, HumanMessage(content=tweet)]}
):
    print(chunk.content, end='')
    reflection += chunk.content

Your tweet has a great foundation with its enthusiasm and engagement prompts! Here are some suggestions to enhance its depth, style, and overall impact:

1. **Add Context**: Mention specific locations or dates to give your audience a clearer picture of what to expect. This can also spark excitement about particular matches or events.

2. **Incorporate Numbers or Stats**: Adding a fun fact or statistic about the World Cup can pique interest. For example, mentioning how many teams will compete or the last time the tournament was held in North America can add depth.

3. **Use a Question to Engage**: Instead of just asking which team they support, consider a more thought-provoking question. For instance, “Which underdog team do you think will surprise everyone?” This invites more detailed responses and engages a wider audience.

4. **Include a Call to Action**: Encourage followers to engage with your tweet by sharing their favorite World Cup memories or predictions. This can create a sense

In [14]:
for chunk in generate_chain.stream(
    {'messages': [request, AIMessage(content=tweet), HumanMessage(content=reflection)]}
):
    print(chunk.content, end='')

🌍⚽️ The countdown to #FIFAWorldCup26 is ON! 🏆 Kicking off in [insert dates], North America is ready to host the world's biggest soccer celebration! 🇺🇸🇨🇦🇲🇽 

Which team are you cheering for? 🌟 Do you think an underdog will steal the spotlight this year? Share your predictions and favorite World Cup memories below! Let’s get the conversation started! 👇 #WorldCup2026 #SoccerFever

(Feel free to insert specific dates for added context!)

In [15]:
from typing import List, Sequence
from langgraph.graph import END, MessageGraph

In [16]:
# defining a function for the generation node
def generation_node(state: Sequence[BaseMessage]):
    return generate_chain.invoke({'messages': state})

# defining a function for the reflection node
def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # messages we need to adjust
    cls_map = {'ai': HumanMessage, 'human': AIMessage}
    # First message is the original user request. We keep it the same for all nodes
    translated = [messages[0]] + [
    cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = reflect_chain.invoke({'messages': translated})
    # We treat the output (AI message) of this as human feedback for the generator
    return HumanMessage(content=res.content)

# initializing the MessageGraph and adding two nodes to the graph: generate and reflect.
builder = MessageGraph()
builder.add_node('generate', generation_node)
builder.add_node('reflect', reflection_node)

# setting the generate node as the starting point
builder.set_entry_point('generate')

MAX_ITERATIONS = 5
def should_continue(state: List[BaseMessage]):
    if len(state) > MAX_ITERATIONS:
        return END
    return 'reflect'

# adding a conditional edge to the graph
builder.add_conditional_edges('generate', should_continue)
builder.add_edge('reflect', 'generate')

# compiling the graph
graph = builder.compile()

In [17]:
inputs = HumanMessage(content='Generate a tweet about FIFA World Cup 26')
response = graph.invoke(inputs)

In [18]:
from IPython.display import Markdown, display

for resp in response:
    display(Markdown(resp.content))
    print('\n' + '-' * 100 + '\n')

Generate a tweet about FIFA World Cup 26


----------------------------------------------------------------------------------------------------



🌍⚽️ The countdown to #FIFAWorldCup26 is ON! Excitement is building as we prepare for the world's biggest soccer celebration across North America! 🏆✨ Who do you think will lift the trophy? Share your predictions! 👇 #WorldCup2026 #SoccerLove


----------------------------------------------------------------------------------------------------



Your tweet is a great start! It captures the excitement surrounding the upcoming FIFA World Cup and invites engagement from your audience. Here are some suggestions to enhance its depth, style, and overall impact:

1. **Add a Historical Context**: Mentioning past World Cup moments or records could enhance the depth of your tweet. This could spark nostalgia and engage followers who appreciate the history of the tournament.

2. **Include a Personal Touch**: Sharing a personal favorite memory or player from past World Cups can make your tweet more relatable and engaging. 

3. **Use a Poll**: Instead of just asking for predictions, consider including a quick poll. This could encourage more interaction and make it easier for followers to share their opinions.

4. **Visual Element**: While you can't include images in the text itself, suggesting a related image or video could enhance engagement. A striking World Cup image or a clip of memorable moments could elevate your post.

5. **Call-to-Action Expansion**: Instead of just asking for predictions, you could prompt followers to share their favorite World Cup moments or players. This invites a broader conversation.

Here’s a revised version of your tweet incorporating these suggestions:

🌍⚽️ The countdown to #FIFAWorldCup26 is ON! As we gear up for the world's biggest soccer celebration across North America, I can't help but reminisce about [insert favorite World Cup memory]! 🏆✨ Who do you think will lift the trophy this time? Vote below or share your all-time favorite World Cup moment! 👇 #WorldCup2026 #SoccerLove

[Insert Poll Options or an engaging visual]

This approach not only builds excitement but also fosters a community dialogue around the World Cup. Keep up the great work!

...


----------------------------------------------------------------------------------------------------



This revised tweet looks fantastic! You've effectively incorporated the suggestions to deepen the engagement and excitement around the FIFA World Cup. Here are a few more tweaks and suggestions to make it even more compelling:

1. **Emphasize the Community Aspect**: You can further enhance the sense of community by encouraging retweets or comments. This can help spread the conversation wider.

2. **Add a Hashtag for the Poll**: Consider using a specific hashtag for your poll to track the engagement better and encourage followers to use it when they share their thoughts. 

3. **Make the Poll Choices More Descriptive**: Instead of just "Dark Horse," you could specify potential dark horse teams to give followers a clearer choice. 

4. **Closing Line**: A closing line that ties back to the excitement can leave a lasting impression.

Here’s a revised version with these suggestions in mind:

🌍⚽️ The countdown to #FIFAWorldCup26 is ON! As we gear up for the world's biggest soccer celebration in North America, I can't help but reminisce about that unforgettable 2014 final! 🏆✨ Who do you think will lift the trophy this time? Vote below or share your all-time favorite World Cup moment! 👇 

🗳️ Who's your pick for the champion? 
1️⃣ Brazil 
2️⃣ France 
3️⃣ Argentina 
4️⃣ Surprise Team (e.g., Netherlands, Portugal, etc.)

#WorldCup2026 #SoccerLove #WC2026Poll 

[📸 Insert an iconic World Cup image or video clip] 

Let’s create unforgettable memories together! 🌟 Can’t wait to see your picks and stories!

This structure would not only improve engagement but also encourage your followers to interact more broadly. Great job! Keep it up!


----------------------------------------------------------------------------------------------------



🌍⚽️ The countdown to #FIFAWorldCup26 is ON! As we gear up for the world's biggest soccer celebration in North America, I can't help but reminisce about that unforgettable 2014 final! 🏆✨ Who do you think will lift the trophy this time? Vote below or share your all-time favorite World Cup moment! 👇 

🗳️ Who's your pick for the champion? 
1️⃣ Brazil 
2️⃣ France 
3️⃣ Argentina 
4️⃣ Surprise Team (e.g., Netherlands, Portugal, etc.)

#WorldCup2026 #SoccerLove #WC2026Poll 

[📸 Insert an iconic World Cup image or video clip] 

Let’s create unforgettable memories together! 🌟 Can’t wait to see your picks and stories! Retweet to join the conversation! 🗣️💬


----------------------------------------------------------------------------------------------------

